In [1]:
from pyagnps import aims
from pathlib import Path

Please install GDAL (see install_gdal.py). The subannagnps module will not work without it
Please install GDAL (see install_gdal.py). The topagnps module will not work without it


In [2]:
# Database credentials
credentials = Path("../../inputs/db_credentials.json")

In [3]:
# CMIP5 Data
# path_to_cmip5_data_dir = Path("D:/AIMS/Datasets/Climate/CMIP/CMIP5/MACAv2METDATA/CNRM-CM5/r1i1p1")
# path_to_cmip5_station_points_clim_id = path_to_cmip5_data_dir / "cmip5_maca_v2_metdata_pts_clim_ids.gpkg"

In [4]:
# Those are optional, you can not specify them and the program will download them from the repository
path_to_nldas2_centroids = Path("D:/AIMS/Datasets/Climate/NLDAS2/NLDAS2_GRID_CENTROIDS_epsg4326.gpkg")
path_to_scs_storm_types  = Path("D:/AIMS/Datasets/Climate/TR-55/scs_storm_types.gpkg")
path_to_precip_zones     = Path("D:/AIMS/Datasets/Management/RUSLE2_Climate/RUSLE2-Climate-Data-20231117/precip_zones_RUSLE2_cleaned_manually_extrapolated_pchip_linear_US_units.gpkg")

In [3]:
output_folder = Path("C:/Users/Luc/Desktop/thuc_test")

In [4]:
w = aims.AIMSWatershed(
                credentials,
                watershed_name="Goodwin Creek",
                watershed_description="As generated by AIMS",
                watershed_location="Mississippi",
                outlet_x=-89.91651,
                outlet_y=34.23151,
                # thuc_id="1148", reach_id=2,
                start_date="2000-01-01",
                end_date="2022-12-31",
                # selected_reaches_for_output=["OUTLET"],
                climate_method="nldas2_database",
                climate_table="climate_nldas2",
                # path_to_cmip_dir=path_to_cmip5_data_dir,
                # path_to_cmip_station_points_id=path_to_cmip5_station_points_clim_id,
                # path_to_nldas2_centroids=path_to_nldas2_centroids,
                # path_to_scs_storm_types=path_to_scs_storm_types,
                # path_to_precip_zones=path_to_precip_zones,
                output_folder=output_folder,
                export_gis=False)

Modifying the output parameters for pre-runs

In [5]:
w.output_options_global_dict['Glbl_All_EV_Nutr']   = ''
w.output_options_global_dict['Glbl_All_EV_Pest']   = ''
w.output_options_global_dict['Glbl_All_EV_Sed']    = ''
w.output_options_global_dict['Glbl_All_EV_Wtr']    = ''
w.output_options_global_dict['V1_EV']              = ''

w.output_options_global_dict['Glbl_All_AA_Sed']    = 'F'
w.output_options_global_dict['Glbl_All_AA_Wtr']    = 'F'
w.output_options_global_dict['Glbl_All_AA_Nutr']   = 'F'
w.output_options_global_dict['Glbl_All_AA_Pest']   = 'F'
w.output_options_global_dict['V2_Concepts']        = 'F'

# w.output_options_global_dict['Reach_Routing_Flag'] = 'F'

w.output_options_aa_dict['AA_Sed_Eros_UA']         = 'T'
w.output_options_aa_dict['AA_Sed_Yld_UA']          = 'T'
w.output_options_aa_dict['AA_Wtr_Yld_UA']          = 'T'

w.output_options_tbl_dict['Gaging_Station_Hyd']    = 'F'
w.output_options_tbl_dict['CCHE1D']                = 'F'

w.simulation_period_data_dict['Number_Init_Years'] = 2

In [6]:
w.generate_annagnps_watershed_input_files()

Loading static layers...
Querying database...
Writing Climate Data...


100%|██████████| 4/4 [00:06<00:00,  1.62s/it]


Writing input files...
Computing watershed global parameters...
Writing control files...
Finished in 40.04 seconds
Output folder: C:\Users\Luc\Desktop\thuc_test


In [11]:
mini_watersheds = w.fragment_watershed(share_global_watershed_climate_params=True, # False to recompute climate parameters for every reach
                     num_processes=16)

Processing reaches:   0%|          | 0/75 [00:00<?, ?it/s]

Fragmentation of watershed took 107.91 seconds
Fragmented watersheds saved in C:\Users\Luc\Desktop\thuc_test\mini_watersheds


In [24]:
# mini_watersheds.sort(key=lambda x: x.name.split("reach_")[1], reverse=False)
mini_watersheds = sorted(mini_watersheds, key=lambda x: int(x.name.split("_")[1]))

In [9]:
df_mini_watersheds = aims.generate_df_mini_watersheds_dirs(mini_watersheds, root_folder=output_folder / "mini_watersheds")
# df_mini_watersheds.to_csv(output_folder / "mini_watersheds" / "dir_list.csv", index=False, header=False)